In [12]:
# input: task_DAG_batch1目录下的所有json文件
# output: 1. 把所有成功的轨迹直接添加到task_DAG_batch1目录下json文件的successful_topo字段中
#         2. 再把successful_topo字段不为空的复制到DAG_with_succ_topo_batch1目录下
# ----------------------------------------------------------------------------------------
import json
trej_cnt = 0
DAG_cnt = 0
import os
for filename in os.listdir('D:/data/[explicit_app]task_DAG_batch1'):
    if filename.endswith('.json') and filename[:-5].isdigit():
        with open(os.path.join('D:/data/[explicit_app]task_DAG_batch1', filename), 'r', encoding='utf-8') as f:
            response = json.load(f)
    else:
        continue
    # response ==  {'task_instruction': "Open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx', select all data, and copy it. Then, paste the text from the clipboard, translate it into 'German', and copy the result. Open 'C:\\Users\\user\\Desktop\\office\\pet introduction.docx', paste the translated text from the clipboard, and save it. Meanwhile, open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx', select the 'A' column, and center the content. Finally, open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx' and export it as a PDF document named 'C:\\Users\\user\\Desktop\\pdf\\The Evolution of Urbanization Rate.pdf'.", 'code': "def execute_task():\n    # Step 1: Copy all data from the Excel file\n    table_in_clipboard, xlsx_in_processing = copy_all_data_from_xlsx('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx')\n\n    # Step 2: Translate the text in the clipboard to German\n    text_in_clipboard = translate_clipboard_to_language(table_in_clipboard, 'German')\n\n    # Step 3: Paste the translated text into the Word document\n    docx_path, text_in_clipboard = paste_clipboard_to_docx_file('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\pet introduction.docx', text_in_clipboard)\n\n    # Step 4: Center the content of column A in the Excel file\n    xlsx_in_processing = open_and_center_xlsx_column_content('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx', 'A')\n\n    # Step 5: Export the Excel file as a PDF\n    pdf_path, xlsx_in_processing = xlsx_to_pdf_export('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx', 'C:\\\\Users\\\\user\\\\Desktop\\\\pdf\\\\The Evolution of Urbanization Rate.pdf')\n\nexecute_task()", 'dag': {'nodes': ['25e2a51e-c019-1a9a-0747-d6fe0e9d457d', '0a4772f6-61ad-49e5-b15e-c02963259484', '8504cc45-066e-d203-cb82-4039ee74ae84', 'a7310aa0-b194-77e3-5c36-996391a1bc7d', 'df3fc68b-fa76-4e19-7da6-aef17792523b'], 'edges': {'25e2a51e-c019-1a9a-0747-d6fe0e9d457d': ['0a4772f6-61ad-49e5-b15e-c02963259484', 'a7310aa0-b194-77e3-5c36-996391a1bc7d', 'df3fc68b-fa76-4e19-7da6-aef17792523b'], '0a4772f6-61ad-49e5-b15e-c02963259484': ['8504cc45-066e-d203-cb82-4039ee74ae84'], '8504cc45-066e-d203-cb82-4039ee74ae84': [], 'a7310aa0-b194-77e3-5c36-996391a1bc7d': [], 'df3fc68b-fa76-4e19-7da6-aef17792523b': []}}}
    # 根据邻接表构建有向无环图并输出拓扑序
    from collections import defaultdict, deque

    # 从response中获取图的信息
    nodes = response['dag']['nodes']
    edges = response['dag']['edges']

    # 构建邻接表和入度表
    graph = defaultdict(list)
    in_degree = defaultdict(int)

    for node in nodes:
        in_degree[node] = 0
        
    for start, end_list in edges.items():
        for end in end_list:
            graph[start].append(end)
            in_degree[end] += 1

    # 使用Kahn算法找出所有拓扑序
    def all_topological_sorts(graph, in_degree, nodes):
        result = []
        
        def backtrack(curr_sequence, remaining_in_degree, available_nodes):
            if len(curr_sequence) == len(nodes):
                result.append(curr_sequence[:])
                return
                
            for node in nodes:
                if remaining_in_degree[node] == 0 and node not in curr_sequence:
                    # 选择当前节点
                    curr_sequence.append(node)
                    
                    # 更新邻居节点的入度
                    neighbor_degrees = {}
                    for neighbor in graph[node]:
                        neighbor_degrees[neighbor] = remaining_in_degree[neighbor]
                        remaining_in_degree[neighbor] -= 1
                    
                    backtrack(curr_sequence, remaining_in_degree, available_nodes)
                    
                    # 回溯
                    curr_sequence.pop()
                    for neighbor, degree in neighbor_degrees.items():
                        remaining_in_degree[neighbor] = degree
        
        backtrack([], in_degree.copy(), set(nodes))
        return result

    # topo序中所有subtask都要有成功轨迹
    with open('D:/data/succ_subtask_traj_ids.json', 'r', encoding='utf-8') as f:
        successful = json.load(f)
    successful_subtasks = []
    for j in successful:
        successful_subtasks.append(j[:-3])

    # # topo序中所有subtask都要有准确评估函数
    # with open('D:/data/succ_subtask_evals.json', 'r', encoding='utf-8') as f:
    #     successful = json.load(f)
    # successful_subtasks = []
    # for j in successful:
    #     successful_subtasks.append(j)

    # # topo序中所有subtask都要有准确评估和成功轨迹
    # with open('D:/data/succ_subtask_both.json', 'r', encoding='utf-8') as f:
    #     successful = json.load(f)
    # successful_subtasks = []
    # for j in successful:
    #     successful_subtasks.append(j)

    # 获取所有拓扑序并保存为列表
    topological_sorts = all_topological_sorts(graph, in_degree, nodes)
    all_sorts = []
    for sort in topological_sorts:
        # 检查当前拓扑序中的所有subtask是否都在successful_subtasks中
        if all(subtask in successful_subtasks for subtask in sort):
            all_sorts.append(sort)
    if all_sorts != []:
        trej_cnt += len(all_sorts)
        DAG_cnt += 1
        response["successful_topo"] = all_sorts
        with open(f'D:/data/[explicit_app]DAG_with_traj_topo_batch1/{filename}', 'w', encoding='utf-8') as f:
            json.dump(response, f, indent=4, ensure_ascii=False)
        # print(f"{i}: {all_sorts}")
    # break
print(trej_cnt, DAG_cnt)



295939 16890


In [ ]:
import os
import json
from tqdm import tqdm
import concurrent.futures
import math

def extract_action(control_text, function, args):
    action = "no operation"
    if function == "click_input":
        times = "Double" if args['double'] else "Single"
        action = f"{times}-click the '{control_text}' with the {args['button']} mouse button."
    elif function == "Keyboard_input":
        keys = args['keys']
        keys = keys.replace("+", "SHIFT+")
        keys = keys.replace("^", "CTRL+") 
        keys = keys.replace("%", "ALT+")
        action = f"Input '{keys}' using the keyboard."
    elif function == "wheel_mouse_input":
        direction = "up" if args['wheel_dist'] > 0 else "down"
        dist = abs(args['wheel_dist'])
        action = f"Scroll the mouse wheel {direction} by {dist} distance."

    return action

def process_file(file_path, dic):
    messages = []
    with open(file_path, 'r', encoding='utf-8') as f:
        dag_data = json.load(f)
        if 'successful_topo' in dag_data:
            for topo in dag_data['successful_topo']:
                history_actions = []
                task_instruction = dag_data["task_instruction"]
                task_intent = dag_data["task_intent"]
                for j in topo:
                    obs = dic[j]["observations"]
                    act = dic[j]["actions"]
                    traj_len = len(obs)
                    for k in range(traj_len):
                        action = {k: v for k, v in act[k].items() if k != 'thought' or k != 'obs'}
                        action["rect"] = [action["rect"][0]*1000//1920, action["rect"][1]*1000//1080, action["rect"][2]*1000//1920, action["rect"][3]*1000//1080]
                        try:
                            obs_content = act[k]["obs"]
                            thought = act[k]["thought"]
                        except KeyError:
                            print(j)
                            assert False
                        m = {
                            "messages": [
                            {
                                "content": """You are a GUI agent. You are given a task and a screenshot of the screen. You need to perform a series of actions to complete the task. Each action contains three fields:
- function: The type of action ('click_input', 'keyboard_input', or 'wheel_mouse_input')
- args: The parameters for the action:
  • click_input args: {button: 'left'/'right'/'middle', double: true/false}
  • keyboard_input args: {keys: string for keyboard input with special formats:
    - Normal text input: 'hello world'
    - Special modifiers: '^' for Ctrl, '+' for Shift, '%' for Alt (e.g. '^a' for Ctrl+A, '^c' for Ctrl+C)
    - Special keys: {DELETE}, {ENTER}, {ESC}, {TAB}
    - Repeated keys: {TAB 2} to press TAB twice
    - Combined usage: '^+{ESC}' for Ctrl+Shift+Esc}
  • wheel_mouse_input args: {wheel_dist: integer for scroll distance (positive for scrolling up, negative for scrolling down)}
- rect: The bounding box coordinates [x1, y1, x2, y2] that define the target region where the action should be performed""",
                                "role": "system"
                            },
                            {
                                "content": f"""<image>
Please generate the next move according to the ui screenshot, instruction and previous actions.
Instruction: {task_instruction}
Previous actions: {history_actions}""",
                                "role": "user"
                            },
                            {
                                "content": f"""Observation: {obs_content}
Thought: {thought}
Low-level Instruction: {extract_action(act[k]["control_text"], act[k]["function"], act[k]["args"])}
Action: {action}""",
                                "role": "assistant"
                            }
                            ],
                            "images": [
                            obs[k]
                            ]
                        }
                        messages.append(m)
                        history_actions.append(extract_action(act[k]["control_text"], act[k]["function"], act[k]["args"]))
    return messages

# Load subtasks
with open("./succ_subtask_trajs.json", 'r', encoding='utf-8') as f:
    subtasks = json.load(f)
dic = {}
for i in subtasks:
    dic[i["subtask_id"]] = i

dag_dir = "./[explicit_app]DAG_with_traj_topo_batch1"
json_files = [f for f in os.listdir(dag_dir) if f.endswith('.json') and f[:-5].isdigit()]

# Split files into 10 chunks
chunk_size = math.ceil(len(json_files) / 10)
file_chunks = [json_files[i:i + chunk_size] for i in range(0, len(json_files), chunk_size)]

# Process each chunk sequentially, but files within chunk in parallel
for chunk_idx, file_chunk in enumerate(file_chunks, 1):
    chunk_messages = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        future_to_file = {
            executor.submit(process_file, os.path.join(dag_dir, filename), dic): filename 
            for filename in file_chunk
        }
        
        for future in tqdm(concurrent.futures.as_completed(future_to_file), 
                          total=len(file_chunk),
                          desc=f"Processing chunk {chunk_idx}/10"):
            chunk_messages.extend(future.result())
    
    # Save chunk results
    output_path = f"./tuning_trajs_{chunk_idx}.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(chunk_messages, f, ensure_ascii=False, indent=4)
